## Data Processing for Adding the Urban Percentage to the PLACE Dataset

This part is used to join the PLACE data with level of Urbanization index:

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import time
import datetime
import math
import pandas as pd
import plotly.express as px
import re

### 1.1 Preprocess CDC Places Data:

In [11]:
cdc_places = pd.read_csv(r'/mnt/d/OneDrive - University College London/T1/CASA0007/0. Research Project/cleaned_data/combined_df.csv', encoding = "UTF-8")

In [12]:
cdc_places['GeoLocation'] = cdc_places['Location'].str.replace('_', ' | ')



[Credit goes to](https://github.com/ChuckConnell/articles/blob/master/fips2county.tsv)


In [15]:
state_county_lookup = pd.read_csv(r'/mnt/d/OneDrive - University College London/T1/CASA0007/0. Research Project/cleaned_data/fips2county.tsv',  sep='\t', header=0)
fips_lookup  = state_county_lookup[['STATE_COUNTY', 'CountyFIPS']].set_index('STATE_COUNTY').to_dict()['CountyFIPS']

In [16]:
state_county_lookup['CountyFIPS']

0        1001
1        1003
2        1005
3        1007
4        1009
        ...  
3138    56037
3139    56039
3140    56041
3141    56043
3142    56045
Name: CountyFIPS, Length: 3143, dtype: int64

In [9]:
from county_matcher import fips_match

In [17]:
for index, row in cdc_places.iterrows():
    for key in fips_lookup:
        if fips_match(key,row['GeoLocation']):
            matched_key = key
            break
    print(matched_key)
    print(row['GeoLocation'])
        # store the matched key in the dataframe
    cdc_places.loc[index, 'GeoLocation'] = matched_key
    cdc_places.loc[index, 'GEOID'] = fips_lookup[matched_key]
    

AK | ALEUTIANS EAST BOROUGH
AK | Aleutians East
AK | ALEUTIANS WEST CENSUS AREA
AK | Aleutians West
AK | ANCHORAGE MUNICIPALITY
AK | Anchorage
AK | BETHEL CENSUS AREA
AK | Bethel
AK | BRISTOL BAY BOROUGH
AK | Bristol Bay
AK | CHUGACH CENSUS AREA
AK | Chugach
AK | COPPER RIVER CENSUS AREA
AK | Copper River
AK | DENALI BOROUGH
AK | Denali
AK | DILLINGHAM CENSUS AREA
AK | Dillingham
AK | FAIRBANKS NORTH STAR BOROUGH
AK | Fairbanks North Star
AK | HAINES BOROUGH
AK | Haines
AK | HOONAH-ANGOON CENSUS AREA
AK | Hoonah-Angoon
AK | JUNEAU CITY AND BOROUGH
AK | Juneau
AK | KENAI PENINSULA BOROUGH
AK | Kenai Peninsula
AK | KETCHIKAN GATEWAY BOROUGH
AK | Ketchikan Gateway
AK | KODIAK ISLAND BOROUGH
AK | Kodiak Island
AK | KUSILVAK CENSUS AREA
AK | Kusilvak
AK | LAKE AND PENINSULA BOROUGH
AK | Lake and Peninsula
AK | MATANUSKA-SUSITNA BOROUGH
AK | Matanuska-Susitna
AK | NOME CENSUS AREA
AK | Nome
AK | NORTH SLOPE BOROUGH
AK | North Slope
AK | NORTHWEST ARCTIC BOROUGH
AK | Northwest Arctic
AK | PET

In [19]:
cdc_places['GEOID'] = cdc_places['GEOID'].astype('int64')
# Convert FIPS codes in DataFrame to string and zero-pad to 5 characters
cdc_places['GEOID'] = cdc_places['GEOID'].apply(lambda x: str(x).zfill(5))

In [20]:
cdc_places

,All teeth lost among adults aged >=65 years,Cervical cancer screening among adult women aged 21-65 years,"Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50-75 years",Mammography use among women aged 50-74 years,"Older adult men aged >=65 years who are up to date on a core set of clinical preventive services: Flu shot past year, PPV shot ever, Colorectal cancer screening","Older adult women aged >=65 years who are up to date on a core set of clinical preventive services: Flu shot past year, PPV shot ever, Colorectal cancer screening, and Mammogram past 2 years",Sleeping less than 7 hours among adults aged >=18 years,Visits to dentist or dental clinic among adults aged >=18 years,Location,Any disability among adults aged >=18 years,...,No leisure-time physical activity among adults aged >=18 years,Obesity among adults aged >=18 years,Physical health not good for >=14 days among adults aged >=18 years,Self-care disability among adults aged >=18 years,Stroke among adults aged >=18 years,Taking medicine for high blood pressure control among adults aged >=18 years with high blood pressure,Vision disability among adults aged >=18 years,Visits to doctor for routine checkup within the past year among adults aged >=18 years,GeoLocation,GEOID
0,16.95,68.35,57.35,59.95,28.45,28.15,34.20,48.50,AK_Aleutians East,28.80,...,29.00,31.70,11.40,3.85,3.40,62.30,6.15,60.70,AK | ALEUTIANS EAST BOROUGH,02013
1,12.05,69.95,62.40,59.50,32.55,30.50,33.95,55.55,AK_Aleutians West,24.20,...,25.05,30.95,9.55,2.90,2.70,59.60,4.50,61.10,AK | ALEUTIANS WEST CENSUS AREA,02016
2,9.75,78.15,68.85,66.25,40.65,32.90,30.90,64.85,AK_Anchorage,25.25,...,20.40,32.95,10.30,2.90,2.60,60.45,4.05,62.80,AK | ANCHORAGE MUNICIPALITY,02020
3,28.05,72.45,56.65,51.25,19.50,18.25,37.05,41.75,AK_Bethel,43.70,...,34.45,42.95,19.45,7.30,5.10,58.65,10.95,61.90,AK | BETHEL CENSUS AREA,02050
4,11.75,79.90,68.10,60.75,33.10,30.00,32.10,59.25,AK_Bristol Bay,28.20,...,20.95,36.00,11.55,3.30,3.30,60.50,4.40,62.80,AK | BRISTOL BAY BOROUGH,02060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3071,13.70,80.05,63.85,60.95,38.65,29.40,30.30,66.40,WY_Sweetwater,29.00,...,25.85,34.45,10.90,3.25,2.70,65.20,4.35,70.35,WY | SWEETWATER,56037
3072,7.95,84.70,67.35,71.70,42.30,37.65,26.50,76.70,WY_Teton,21.25,...,16.30,21.95,8.20,2.35,2.20,63.85,3.10,66.75,WY | TETON,56039
3073,13.00,80.65,63.70,62.00,33.90,29.05,35.10,63.90,WY_Uinta,29.35,...,23.15,33.50,10.95,3.30,2.85,64.95,4.30,64.55,WY | UINTA,56041
3074,12.75,80.40,59.00,63.60,41.05,26.05,30.50,64.45,WY_Washakie,30.10,...,23.85,33.55,11.10,3.40,3.05,68.15,4.50,65.05,WY | WASHAKIE,56043


## 2 Peprocess the County urban vs rural dataset

In [14]:
rural_v_urban_county = pd.read_excel(r'/mnt/d/OneDrive - University College London/T1/CASA0007/0. Research Project/cleaned_data/2020_UA_COUNTY.xlsx', sheet_name = '2020_UA_COUNTY')

In [26]:
rural_v_urban_county

,STATE,COUNTY,STATE_NAME,COUNTY_NAME,POP_COU,HOU_COU,ALAND_COU,ALAND_Mi²_COU,POPDEN_COU,HOUDEN_COU,...,POPPCT_RUR,HOU_RUR,HOUPCT_RUR,ALAND_RUR,ALAND_Mi²_RUR,ALAND_PCT_RUR,POPDEN_RUR,HOUDEN_RUR,RURALBLOCKS,GEOID
0,01,001,Alabama,Autauga,58805,24350,1539634184,594.452758,98.922916,40.962044,...,0.406768,9991,0.410308,1483727020,572.867002,0.963688,41.754892,17.440348,991,01001
1,01,003,Alabama,Baldwin,231767,124148,4117656199,1589.827058,145.781265,78.088997,...,0.375865,40740,0.328157,3762600021,1452.739868,0.913772,59.964624,28.043562,3181,01003
2,01,005,Alabama,Barbour,25223,11618,2292160149,885.003034,28.500467,13.127639,...,0.659200,7538,0.648821,2276027730,878.774307,0.992962,18.920672,8.577857,1011,01005
3,01,007,Alabama,Bibb,22293,9002,1612188717,622.466064,35.814001,14.461833,...,1.000000,9002,1.000000,1612188717,622.466064,1.000000,35.814001,14.461833,1090,01007
4,01,009,Alabama,Blount,59134,24622,1670259090,644.887035,91.696680,38.180330,...,0.904894,22337,0.907197,1658933117,640.514076,0.993219,83.542270,34.873551,2207,01009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3229,72,151,Puerto Rico,Yabucoa,30426,14283,143005188,55.214303,551.052867,258.682972,...,0.180799,2424,0.169712,82813495,31.974290,0.579094,172.044475,75.810908,87,72151
3230,72,153,Puerto Rico,Yauco,34172,16599,175371930,67.711102,504.673516,245.144437,...,0.178158,2856,0.172059,140103916,54.094122,0.798896,112.544576,52.796864,173,72153
3231,78,010,US Virgin Islands,St. Croix,41004,25470,215914981,83.364774,491.862425,305.524729,...,0.064189,1757,0.068983,126886794,48.990991,0.587670,53.724163,35.863737,439,78010
3232,78,020,US Virgin Islands,St. John,3881,3645,50998053,19.690348,197.101643,185.116076,...,0.236279,964,0.264472,42952078,16.583797,0.842230,55.294935,58.129027,264,78020


In [22]:
rural_v_urban_county['STATE'] = rural_v_urban_county['STATE'].apply(lambda x: str(x).zfill(2))
rural_v_urban_county['COUNTY'] = rural_v_urban_county['COUNTY'].apply(lambda x: str(x).zfill(3))

In [24]:
rural_v_urban_county['GEOID'] = rural_v_urban_county['STATE'] + rural_v_urban_county['COUNTY']

|Field Name	| Field Description |
| --- | --- |
STATE | Two digit State Code
COUNTY | Three digit County Code
STATE_NAME | Name of the State
COUNTY_NAME | Name of the County or County equivalent.   (* in COUNTY_NAME indicates value(s) in this row were rounded up from 0.00 to 0.01 or down from 100 to 99.99)
POP_COU | 2020 Census total population of the County
HOU_COU	| 2020 Census total housing unit count of the County
ALAND_COU | 2020 land area of the County (square meters)
ALAND_Mi²_COU | 2020 land area of the County (square miles)
POPDEN_COU |	2020 population density of the County (square miles)
HOUDEN_COU |	2020 housing unit density of the County (square miles)
TOTALBLOCKS_COU |	2020 total blocks within the County
POP_URB |	2020 Census population of the County within Urban blocks
POPPCT_URB |	Percent of the 2020 Census population of the County within Urban blocks
HOU_URB |	2020 Census housing units in the County within Urban blocks
HOUPCT_URB |	Percent of 2020 Census housing units in the County within Urban blocks
ALAND_URB |	2020 Urban land area of the County (square meters)
ALAND_Mi²_URB |	2020 Urban land area of the County (square miles)
ALAND_PCT_URB |	Percent of 2020 land within the County that is classified as Urban
POPDEN_URB |	2020 Urban population density of the County (square miles)
HOUDEN_URB |	2020 Urban housing unit density of the County (square miles)
URBANBLOCKS |	2020 blocks classified as Urban within County
POP_RUR |	2020 Census population of the County within Rural blocks
POPPCT_RUR |	Percent of the 2020 Census population in the County within Rural blocks
HOU_RUR |	2020 Census housing units in the County within Rural blocks
HOUPCT_RUR |	Percent of the 2020 housing units in the County within Rural blocks
ALAND_RUR |	2020 Rural land area of the County (square meters)
ALAND_Mi²_RUR |	2020 Rural land area of the County (square miles)
ALAND_PCT_RUR |	Percent of 2020 land within the County that is classified as Rural
POPDEN_RUR |	2020 Rural population density of the County (square miles)
HOUDEN_RUR |	2020 Rural housing unit density of the County (square miles)
RURALBLOCKS |	2020 blocks classified as Rural within County


Selected data to join:

|Field Name	| Field Description |
| GEOID | Join on this column |

In [ ]:
county_rvu = rural_v_urban_county.loc[:, ['GEOID', 'ALAND_PCT_URB', 'POPPCT_URB', 'HOUPCT_URB','HOUPCT_RUR']] 

## 3 Join two datasets

## 4 Save the combined data

In [ ]:
# Save to this r'/mnt/d/OneDrive - University College London/T1/CASA0007/0. Research Project/cleaned_data/combined/combined_cdc_r_v_u_df.csv'
